In [3]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1"
JULIA_PACKAGES="SymbolicRegression"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"'
  done

fi

In [4]:
!pip3 install -U pysr==0.18.2 pytorch_lightning

In [5]:
import sympy
import numpy as np
from matplotlib import pyplot as plt
from pysr import PySRRegressor
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split

In [6]:
###### np.random.seed(0)
N = 100000
Nt = 100
X = 6 * np.random.rand(N, Nt, 5) - 3
y_i = X[..., 0] ** 2 + 6 * np.cos(2 * X[..., 2])
y = np.sum(y_i, axis=1) / y_i.shape[1]
z = y**2
X.shape, y.shape

((100000, 100, 5), (100000,))

In [2]:
hidden = 100

def mlp(size_in, size_out):
    return nn.Sequential(
        nn.Linear(size_in, hidden),
        nn.ReLU(),
        nn.Linear(hidden, hidden),
        nn.ReLU(),
        nn.Linear(hidden, hidden),
        nn.ReLU(),
        nn.Linear(hidden, size_out))


max_epochs = 2
Xt = torch.tensor(X).float()
zt = torch.tensor(z).float()
hidden = 128
total_steps = 50000


def mlp(size_in, size_out, act=nn.ReLU):
    return nn.Sequential(
        nn.Linear(size_in, hidden),
        act(),
        nn.Linear(hidden, hidden),
        act(),
        nn.Linear(hidden, hidden),
        act(),
        nn.Linear(hidden, size_out),
    )


class SumNet(pl.LightningModule):
    def __init__(self):
        super().__init__()

    ########################################################
    # The same inductive bias as above!
        self.g = mlp(5, 1)
        self.f = mlp(1, 1)

    def forward(self, x):
        y_i = self.g(x)[:, :, 0]
        y = torch.sum(y_i, dim=1, keepdim=True) / y_i.shape[1]
        z = self.f(y)
        return z[:, 0]

    ########################################################

    # PyTorch Lightning bookkeeping:
    def training_step(self, batch, batch_idx):
        x, z = batch
        predicted_z = self(x)
        loss = F.mse_loss(predicted_z, z)
        return loss

    def validation_step(self, batch, batch_idx):
        return self.training_step(batch, batch_idx)

    def configure_optimizers(self):
        self.trainer.reset_train_dataloader()

        optimizer = torch.optim.Adam(self.parameters(), lr=self.max_lr)
        scheduler = {
            "scheduler": torch.optim.lr_scheduler.OneCycleLR(
                optimizer,
                max_lr=self.max_lr,
                total_steps=self.total_steps,
                final_div_factor=1e4,
            ),
            "interval": "step",
        }
        return [optimizer], [scheduler]

class SumNet(pl.LightningModule):
    def __init__(self):
        super().__init__()
        X_train, X_test, z_train, z_test = train_test_split(Xt, zt, random_state=0)
        self.train_set = TensorDataset(X_train, z_train)
        self.test_set = TensorDataset(X_test, z_test)

        # The same inductive bias as above!
        self.g = mlp(5, 1)
        self.f = mlp(1, 1)

    def forward(self, x):
        y_i = self.g(x)[:, :, 0]
        y = torch.sum(y_i, dim=1, keepdim=True) / y_i.shape[1]
        z = self.f(y)
        return z[:, 0]

    ########################################################

    # PyTorch Lightning bookkeeping:
    def training_step(self, batch, batch_idx):
        x, z = batch
        predicted_z = self(x)
        loss = F.mse_loss(predicted_z, z)
        return loss

    def validation_step(self, batch, batch_idx):
        return self.training_step(batch, batch_idx)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)
        scheduler = {'scheduler': torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-2,
                                                                      steps_per_epoch=len(self.train_dataloader()),
                                                                      epochs=max_epochs,
                                                                      final_div_factor=1e4),
                     'interval': 'step'}
        return [optimizer], [scheduler]

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=128, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=256, num_workers=4)


pl.seed_everything(0)
model = SumNet()
trainer = pl.Trainer(max_epochs=max_epochs)
trainer.fit(model)

NameError: name 'torch' is not defined

In [1]:
trainer.save_checkpoint("example.ckpt")

from google.colab import files
files.download('example.ckpt')

NameError: name 'trainer' is not defined

Save and upload model

model = MyLightningModule(hparams)

trainer.fit(model)

trainer.save_checkpoint("example.ckpt")

new_model = MyModel.load_from_checkpoint(checkpoint_path="example.ckpt")

In [9]:
np.random.seed(0)
idx = np.random.randint(0, 10000, size=1000)

X_for_pysr = Xt[idx]
y_i_for_pysr = model.g(X_for_pysr)[:, :, 0]
y_for_pysr = torch.sum(y_i_for_pysr, dim=1) / y_i_for_pysr.shape[1]
z_for_pysr = zt[idx]  # Use true values.

X_for_pysr.shape, y_i_for_pysr.shape

(torch.Size([1000, 100, 5]), torch.Size([1000, 100]))

In [ ]:
np.random.seed(1)
tmpX = X_for_pysr.detach().numpy().reshape(-1, 5)
tmpy = y_i_for_pysr.detach().numpy().reshape(-1)
idx2 = np.random.randint(0, tmpy.shape[0], size=3000)

model = PySRRegressor(
    niterations=20,
    binary_operators=["plus", "sub", "mult"],
    unary_operators=["cos", "square", "neg"],
)
model.fit(X=tmpX[idx2], y=tmpy[idx2])

/usr/local/lib/python3.10/dist-packages/pysr/sr.py:1302: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Compiling Julia backend...


In [ ]:
model